In [ ]:
!pip install --quiet datasets

In [ ]:
from datasets import load_dataset

data = load_dataset("wikipedia", "20220301.simple")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
data['train'][6]

{'id': '13',
 'url': 'https://simple.wikipedia.org/wiki/Alan%20Turing',
 'title': 'Alan Turing',
 'text': 'Alan Mathison Turing OBE FRS (London, 23 June 1912 – Wilmslow, Cheshire, 7 June 1954) was an English mathematician and computer scientist. He was born in Maida Vale, London.\n\nEarly life and family \nAlan Turing was born in Maida Vale, London on 23 June 1912. His father was part of a family of merchants from Scotland. His mother, Ethel Sara, was the daughter of an engineer.\n\nEducation \nTuring went to St. Michael\'s, a school at 20 Charles Road, St Leonards-on-sea, when he was five years old.\n"This is only a foretaste of what is to come, and only the shadow of what is going to be.” – Alan Turing.\n\nThe Stoney family were once prominent landlords, here in North Tipperary. His mother Ethel Sara Stoney (1881–1976) was daughter of Edward Waller Stoney (Borrisokane, North Tipperary) and Sarah Crawford (Cartron Abbey, Co. Longford); Protestant Anglo-Irish gentry.\n\nEducated in Dub

In [ ]:
# Assuming your dataset is stored in a variable called 'data'
text_list = data['train'][:100]['text']
text = ' '.join(text_list)

# load text from another file
#with open("/content/biden-sotu-2023-planned-official.txt", "r") as file:
    #text = file.read()

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW
from torch.utils.data import DataLoader, Dataset
import torch

In [ ]:
# Load model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

# reset model weights to train it from scratch
model.init_weights()

# Preprocess the text into chunks
def preprocess_into_chunks(text, tokenizer, max_length, stride):
    tokens = tokenizer.encode(text)
    chunks = [tokens[i: i + max_length] for i in range(0, len(tokens), max_length - stride)]
    return chunks

max_length = 1024
stride = 200
chunks = preprocess_into_chunks(text, tokenizer, max_length, stride)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Modify the dataset class to include padding
class TextDataset(Dataset):
    def __init__(self, chunks, block_size):
        self.chunks = chunks
        self.block_size = block_size

    def __getitem__(self, idx):
        chunk = self.chunks[idx]
        # Pad the chunk if necessary
        padded_chunk = chunk + [tokenizer.pad_token_id] * (self.block_size - len(chunk))
        return torch.tensor(padded_chunk)

    def __len__(self):
        return len(self.chunks)

# Initialize dataset and dataloader
block_size = max_length  # Ensure this matches the model's max input size
dataset = TextDataset(chunks, block_size)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Training setup
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
model.train()
for epoch in range(150):  # Adjust number of epochs as needed
    for batch in dataloader:
        batch = batch.to(device)
        optimizer.zero_grad()
        outputs = model(batch, labels=batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch}, Loss: {loss.item()}")

Token indices sequence length is longer than the specified maximum sequence length for this model (92659 > 1024). Running this sequence through the model will result in indexing errors


Streaming output truncated to the last 5000 lines.
Epoch 62, Loss: 0.11276597529649734
Epoch 62, Loss: 0.16562096774578094
Epoch 62, Loss: 0.15483790636062622
Epoch 62, Loss: 0.1363372951745987
Epoch 62, Loss: 0.13633324205875397
Epoch 62, Loss: 0.1906842142343521
Epoch 62, Loss: 0.1556892991065979
Epoch 62, Loss: 0.19386035203933716
Epoch 62, Loss: 0.1551906317472458
Epoch 62, Loss: 0.19993101060390472
Epoch 62, Loss: 0.16835099458694458
Epoch 62, Loss: 0.1398066282272339
Epoch 62, Loss: 0.1618780940771103
Epoch 62, Loss: 0.1720147728919983
Epoch 62, Loss: 0.20197539031505585
Epoch 62, Loss: 0.1627621054649353
Epoch 62, Loss: 0.18181312084197998
Epoch 62, Loss: 0.16688720881938934
Epoch 62, Loss: 0.16282670199871063
Epoch 62, Loss: 0.14847558736801147
Epoch 62, Loss: 0.15388613939285278
Epoch 62, Loss: 0.1653853952884674
Epoch 62, Loss: 0.1553652137517929
Epoch 62, Loss: 0.16909483075141907
Epoch 62, Loss: 0.16839203238487244
Epoch 62, Loss: 0.14818237721920013
Epoch 62, Loss: 0.21533

In [ ]:
model.save_pretrained('./gpt2wikipedia100')
tokenizer.save_pretrained('./gpt2wikipedia100')

('./gpt2wikipedia100/tokenizer_config.json',
 './gpt2wikipedia100/special_tokens_map.json',
 './gpt2wikipedia100/vocab.json',
 './gpt2wikipedia100/merges.txt',
 './gpt2wikipedia100/added_tokens.json')

In [ ]:
model_path = '/content/gpt2wikipedia100'  # Path to your saved model
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
def generate_text(prompt_text, model, tokenizer, max_length=50, num_return_sequences=1):
    # Encode the prompt text
    encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors="pt")
    encoded_prompt = encoded_prompt.to(device)

    # Generate text
    output_sequences = model.generate(
        input_ids=encoded_prompt,
        max_length=max_length + len(encoded_prompt[0]),
        temperature=1.0,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=num_return_sequences,
    )

    # Decode the output sequences
    generated_sequences = []
    for generated_sequence_idx, generated_sequence in enumerate(output_sequences):
        generated_sequence = generated_sequence.tolist()
        text = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True)
        generated_sequences.append(text[len(tokenizer.decode(encoded_prompt[0], clean_up_tokenization_spaces=True)):])

    return generated_sequences

In [ ]:
prompt = "Spain is"
generated_texts = generate_text(prompt, model, tokenizer, max_length=256, num_return_sequences=1)

for i, text in enumerate(generated_texts):
    print(f"Generated text {i+1}: {text}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated text 1:  a Christian country. Most of Argentina's people (80 percent) are Roman Catholic. Argentina also has the largest population of Jewish community after Israel and US. Middle Eastern immigrants who were Muslims converted to Catholicism, but there are still Muslims as well; Arabs, Druze(er), Turkmen and Pashai Muslims.

Medicine is socialized and so is education, making Argentina's literacy rate about 98%. State University is free as well.

Related pages
Argentina at the Olympics 1896 - 2000
Argentinians national football team
List of rivers of Argentina

References 

 General information system Atletico de la Nación Official website
 Gobierno Electrónico por favorited en Espírito Santo 2012
 Honorable Senado: Books or los Blancos informados con el mundial y astroturfing? Secretaría comunidad 1825 estándián ("Essay on Spanish Language") [Spanish-only pamphlet with many references to official government lists of countries in Latin America and Europe at the 2010 United Nati